In [ ]:
# default_exp core

In [ ]:
#!sh setup.sh

In [2]:
#export

import json
import re


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [4]:
!pip install biopython pydash

     |████████████████████████████████| 2.3 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 769 kB/s  eta 0:00:01


In [5]:
#export
import pandas as pd
from Bio import Entrez
from pydash import get

def get_pubmed_records(pmids):
    Entrez.email = "strasser.ms@gmail.com"
    # handle type is http.client.HTTPResponse
    handle = Entrez.efetch(
        db="pubmed",
        id=",".join(pmids),
        #api_key="be67e0a1be023d17fff5334a6c6f45287a08",
        rettype="xml",
        retmode="text",
    )
    records = Entrez.read(handle)
    return records

def get_attribute_text(AbstractText, NlmCategory):
    if AbstractText is None:
        return None
    for string_element in AbstractText:
        cat = ""
        try:
            #the worst API arbitrary nonsense ... 1h of my life for getting data out of it
            cat = string_element.attributes["NlmCategory"]
        except:
            None
        
        if cat == NlmCategory:
            return string_element

def get_segment(record, segment_label):
    if record is None:
        return None
    return str(
        get_attribute_text(
            get(record, "MedlineCitation.Article.Abstract.AbstractText"), 
            segment_label
        )
    )

In [6]:
#export
import itertools
from pydash import find_index
#OLD one-off functions that need refactoring
def sentence_has_phrase(sentence, cue_phrases):
    for c in cue_phrases: #unsafe scope
        if c in sentence:
            return True
    return False
    
def find_sentence_in_abstract(paragraph, bias=0):
    """checks paragraph for key sentences. Returns first matching hit"""
    p_sents = split_into_sentences(paragraph)
    idx = find_index(p_sents, sentence_has_phrase)
    if idx < 0:
        return ""
    
    i = idx + bias #bias- for sentence before or after
    if i < 0:
        return ""
    return get(p_sents, i)

In [7]:
import requests as req

# url = "https://github.com/derekchuank/high-frequency-vocabulary/blob/master/30k.txt"
# res = req.get(url)
# vocab30k = res.text.split("\t\n")
# vocab30k[:5]

In [8]:
#export

import spacy
import en_core_web_md
with open("vocab30k.txt") as f:
    vocab30k = f.read().split("\t\n")

nlp = en_core_web_md.load()

extra_vocab = [
               "-PRON-", "-pron-", ".", ",", ";"  #lemmatizer inconsitency
]

special_tokens = {
    "(": "-lrb-",
    ")": "-rrb-"
}

def replace_outof_vocab_words(text, vocab, nlp=nlp, extra_vocab=extra_vocab, special_tokens=special_tokens):
    vocab += extra_vocab
    newtext = ""
    for token in nlp(text):
        t = token.text
        if special_tokens.get(t):
            t=special_tokens.get(t)
        elif token.lemma_.lower() not in vocab and token.tag_ is not None:
            t= "ii" + token.tag_.lower()
        newtext += t + " "

    return newtext

ModuleNotFoundError: No module named 'spacy'

In [ ]:
#replace_outof_vocab_words('Pancreaticobiliary maljunction (PBM) refers to the union of the pancreatic and biliary ducts outside of the duodenal wall.')

In [9]:
#NLTK
patterns = [
    (r'.*ing$', 'VBG'),               # gerunds
    (r'.*ed$', 'VBD'),                # simple past
    (r'.*es$', 'VBZ'),                # 3rd singular present
    (r'.*ould$', 'MD'),               # modals
    (r'.*\'s$', 'NN$'),               # possessive nouns
    (r'.*s$', 'NNS'),                 # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),  # cardinal numbers
    (r'.*', 'NN')                     # nouns (default)
]

## Pubmed Abstract/Fulltext Fetching and Cleaning

Pubmed does not give fulltext/abstract text in a good format. It gives a CSV with the metadata of searched results. That means you have to use the Entrez fetch and then merge the abstracts and text information with the metadata to have something workable

In [16]:
# with open("clinical_study_abst.txt") as f:
#     text = f.read()
    
df = pd.read_csv("downloads/clin_study_meta.csv")
records = get_pubmed_records([str(pmid) for pmid in df["PMID"]])
len([r for r in records["PubmedArticle"] if r is not None])

articles = records["PubmedArticle"]

g = {
    "PMID": [str(get(r,"MedlineCitation.PMID")) for r in articles],
    "BACKGROUND": [get_segment(r, "BACKGROUND") for r in articles],
    "OBJECTIVE": [get_segment(r, "OBJECTIVE") for r in articles],
    "METHODS": [get_segment(r, "METHODS") for r in articles],
    "RESULTS": [get_segment(r, "RESULTS") for r in articles],
    "CONCLUSIONS": [get_segment(r, "CONCLUSIONS") for r in articles]
}


df2 = pd.DataFrame(g)
df2["PMID"] = df2["PMID"].astype(int)
dfx = pd.merge(df, df2, on="PMID")
dfx.to_csv("enriched_pm_10k.csv")


dfx.head(), len(df), len(df2), df[:6768]["PMID"][0], df2[:6768]["PMID"][0]

(       PMID                                              Title  \
 0  28946522  The effect of manual therapy and exercise in p...   
 1  28325149  Low Dose Naltrexone in the Treatment of Fibrom...   
 2  29514165  High-Dose Erythropoietin for Asphyxia and Ence...   
 3  29424396  [Quality of sleep and academic performance in ...   
 4  29754671  Rationale and design for the Vascular Outcomes...   
 
                                              Authors  \
 0                 Ulger O, Demirel A, Oz M, Tamer S.   
 1  Metyas S, Chen CL, Yeter K, Solyman J, Arkfeld...   
 2  Juul SE, Comstock BA, Heagerty PJ, Mayock DE, ...   
 3  Bugueño M, Curihual C, Olivares P, Wallace J, ...   
 4  Capell WH, Bonaca MP, Nehler MR, Chen E, Kitte...   
 
                                             Citation First Author  \
 0  J Back Musculoskelet Rehabil. 2017 Nov 6;30(6)...      Ulger O   
 1  Curr Rheumatol Rev. 2018;14(2):177-180. doi: 1...     Metyas S   
 2  Neonatology. 2018;113(4):331-338. doi:

In [63]:
#export
def get_first(p):
    return get(split_into_sentences(p), 0)
#get(split_into_sentences(t), 0) for t in dfx["METHODS"][:10] if split_into_sentences(t))

In [77]:
dfy

,PMID,Title,source,DOI


In [88]:
#reformating data
dfy = pd.DataFrame([], columns=["PMID", "Title", "source", "DOI", "text"])
tags = ["OBJECTIVE", "METHODS", "RESULTS", "CONCLUSIONS"]
for tag in tags:
    df = dfx[dfx[tag] != "None"]
    df["paragraph"] = df[tag]
    df["text"] = dfx[tag].map(get_first)
    df["source"] = "pm_" + tag
    df = df[dfy.columns]
    dfy = dfy.append(df)

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [89]:
dfy = dfy.dropna(subset=["text"])
dfy["labels"] = "0"

In [91]:
dfy.to_csv("downloads/struct_abstracts_negatives_18k.csv")

In [30]:
[get(split_into_sentences(t), 0) for t in dfx["METHODS"][:10] if split_into_sentences(t)]

['A total of one-hundred thirteen patients diagnosed as CLBP were enrolled to the study.',
 'Naltrexone is an opioid receptor antagonist used to treat alcohol and opioid dependence.',
 None,
 'Students of the first and second year of high school answered an interview about socio-demographic background, academic performance, student activities and subjective sleep quality; they were evaluated using the Pittsburgh Sleep Quality Index (PSQI).',
 None,
 'University students were randomly allocated into exercise (n: 28) and control (n: 25) groups.',
 'A sample of 181 soon-to-graduate nursing students participated in the study and were assigned at random to one of two therapies: physical exercise (n = 90) or mindfulness meditation (n = 91).',
 None,
 'A randomized study.',
 None]

## Text Features with spacy extensions

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.polarity      # Polarity: -0.125
doc._.subjectivity  # Sujectivity: 0.9
doc._.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]